In [1]:
%pip install shap

import json
import pandas as pd
import numpy as np
import shap
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

Note: you may need to restart the kernel to use updated packages.


2025-05-30 00:05:30.037447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748563530.060879 1184804 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748563530.068301 1184804 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748563530.089969 1184804 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748563530.089992 1184804 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748563530.089995 1184804 computation_placer.cc:177] computation placer alr

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
def evaluate_classification(y_true, y_pred, results, solver_name, label):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    print(f"{label} | Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")

    results.append({
        "Solver": solver_name,
        "Dataset": label,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1-Score": f1
    })

In [ ]:
import shap
import numpy as np
import pandas as pd
import os

def log_shap_and_importance_classification(model, X_val, y_val, pred_val,features, solver_name, target):

    # background in the same shape as X_val
    if X_val.ndim == 3:
        idx = np.random.choice(X_val.shape[0], min(100, X_val.shape[0]), replace=False)
        background = X_val[idx, :, :]   # (b, f, 1)
        shap_input = X_val               # (n, f, 1)
    else:
        # non-CNN: 2D
        idx = np.random.choice(X_val.shape[0], min(100, X_val.shape[0]), replace=False)
        background = X_val[idx, :]       # (b, f)
        shap_input = X_val               # (n, f)

  
    try:
        explainer = shap.GradientExplainer(model, background)
        shap_values = explainer.shap_values(shap_input)
    except Exception as e:
        print(f"SHAP failed for {target}, error: {e}")
        return

    # Normalize to a single numpy array of shape (n, f, 1, c) or list-of-arrays
    if isinstance(shap_values, list):
        # list of arrays for shape (n, f, 1) or (n, f)
        # stack to (c, n, f, 1) or (c, n, f)
        array = np.stack(shap_values, axis=0)
        # now array.shape = (c, n, f[, 1])
        abs_mean = np.mean(np.abs(array), axis=0)  # (n, f[, 1])
    else:
        # shap_values itself is an array, maybe shape (n, f, 1, c) or (n, f, c)
        arr = np.array(shap_values)
        if arr.ndim == 4:
            # (n, f, 1, c) to (n, f, c)
            arr = arr.squeeze(axis=2)
        #  arr is (n, f, c) or (n, f)
        if arr.ndim == 3:
            # average over the classes axis
            abs_mean = np.mean(np.abs(arr), axis=2)  # (n, f)
        else:
            # arr is already (n, f)
            abs_mean = np.abs(arr)

    # abs_mean is (n_samples, n_features)
    n, f = abs_mean.shape
    assert f == len(features), f"Expected {len(features)} features, got {f}"


    shap_df = pd.DataFrame(abs_mean, columns=features)
    shap_df["predicted_value"] = pred_val
    shap_df["actual_value"]    = y_val
    shap_df["target"]          = target
    shap_df["solver"]          = solver_name

    os.makedirs("./cnn_class/cnn_shap_values", exist_ok=True)
    shap_filename = f"./cnn_class/cnn_shap_values/shap_{solver_name}_{target}_classification.csv"
    shap_df.to_csv(shap_filename, index=False)

    
    feature_importance = abs_mean.mean(axis=0) 
    importance_df = pd.DataFrame({
        "feature": features,
        "shap_importance": feature_importance,
        "target": target,
        "solver": solver_name
    })

    os.makedirs("./cnn_class/cnn_feature_importance", exist_ok=True)
    imp_file = "./cnn_class/cnn_feature_importance/cnn_feature_importance_classification.csv"
    importance_df.to_csv(imp_file, mode='a', index=False,
                         header=not os.path.exists(imp_file))

    # Top-5 features
    top5 = importance_df.nlargest(5, "shap_importance")
    top5_file = "./cnn_class/cnn_feature_importance/cnn_top5_feature_importance_classification.csv"
    top5.to_csv(top5_file, mode='a', index=False,
                header=not os.path.exists(top5_file))

    print(f"SHAP saved for {solver_name}-{target}.")


In [5]:
def build_1d_cnn_classifier(input_dim, num_classes=5, learning_rate=0.001):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, Input
    from tensorflow.keras.optimizers import Adam

    model = Sequential([
        Input(shape=(input_dim, 1)),  # Reshape features for Conv1D
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def train_cnn_classifier_for_solver(solver_name, train_file, test_file, val_file):
    print(f"\nSolver: {solver_name}")
    
    df_train = pd.read_csv(train_file).dropna()
    df_test  = pd.read_csv(test_file).dropna()
    df_val   = pd.read_csv(val_file).dropna()

    features = [
        "number_of_elements","capacity","max_weight","min_weight","mean_weight",
        "median_weight","std_weight","weight_range","max_profit","min_profit","mean_profit",
        "median_profit","std_profit","profit_range","renting_ratio","mean_weight_profit_ratio",
        "median_weight_profit_ratio","capacity_mean_weight_ratio","capacity_median_weight_ratio",
        "capacity_std_weight_ratio","std_weight_profit_ratio","weight_profit_correlation",
        "ram","cpu_cores"
    ]
    target_cols = ["solution_time", "optimality_gap", "peak_memory"]

    # Load the solver's bin edges 
    bins_dir = os.path.join(BINS_BASE_DIR, f"{solver_name}_bins")
    if not os.path.isdir(bins_dir):
        raise FileNotFoundError(f"No bins directory: {bins_dir}")
    json_files = [f for f in os.listdir(bins_dir) if f.endswith("_bins.json")]
    if len(json_files) != 1:
        raise FileNotFoundError(f"Expected one json in {bins_dir}, found: {json_files}")
    bin_path = os.path.join(bins_dir, json_files[0])
    with open(bin_path, "r") as f:
        bin_edges_dict = json.load(f)

    # Scale features
    scaler = StandardScaler().fit(df_train[features])
    X_train = scaler.transform(df_train[features])
    X_test  = scaler.transform(df_test[features])
    X_val   = scaler.transform(df_val[features])

    # Reshape for CNN input
    X_train_cnn = X_train.reshape((-1, X_train.shape[1], 1))
    X_test_cnn  = X_test .reshape((-1, X_test .shape[1], 1))
    X_val_cnn   = X_val  .reshape((-1, X_val  .shape[1], 1))

   
    os.makedirs("./cnn_class/cnn_classifier_models", exist_ok=True)
    results = []

    for target in target_cols:
        if target not in bin_edges_dict:
            print(f" No bins for '{target}'")
            continue
        edges = bin_edges_dict[target]

        # Labels to 0..n_bins-1
        def to_bins(arr):
            labels = np.digitize(arr, edges[:-1], right=False) - 1
            return np.clip(labels, 0, len(edges) - 2)

        y_train = to_bins(df_train[target].values)
        y_test  = to_bins(df_test [target].values)
        y_val   = to_bins(df_val  [target].values)

        
        max_train = y_train.max()
        y_test = np.clip(y_test, 0, max_train)
        y_val  = np.clip(y_val, 0, max_train)

        
        if len(np.unique(y_train)) < 2:
            print(f"  skipping '{target}': only one class in train")
            continue

        
        num_classes = int(max_train) + 1

        # one‐hot encode
        y_train_cat = to_categorical(y_train, num_classes=num_classes)
        y_val_cat   = to_categorical(y_val,   num_classes=num_classes)
        y_test_cat  = to_categorical(y_test,  num_classes=num_classes)

        # debug 
        print(f"\n=== {target} → num_classes={num_classes} ===")
        print(" train bins:", np.unique(y_train))
        print("  val bins:", np.unique(y_val))
        print(" test bins:", np.unique(y_test))

        # Train & evaluate for each epoch 
        for epochs in [50, 100]:
            print(f"\nTraining CNN for '{target}' ({epochs} epochs)")
            model = build_1d_cnn_classifier(
                input_dim=X_train.shape[1],
                num_classes=num_classes,
                learning_rate=0.001
            )
            history = model.fit(
                X_train_cnn, y_train_cat,
                validation_data=(X_val_cnn, y_val_cat),
                epochs=epochs,
                batch_size=64,
                verbose=0
            )

            # predict
            pred_test_probs = model.predict(X_test_cnn)
            pred_val_probs  = model.predict(X_val_cnn)
            best_pred_test  = np.argmax(pred_test_probs, axis=1)
            best_pred_val   = np.argmax(pred_val_probs,  axis=1)

            print("[TEST]")
            evaluate_classification(y_test, best_pred_test, results,
                                    solver_name, f"{target} (Test) - {epochs}e")
            print("[VAL]")
            evaluate_classification(y_val, best_pred_val,  results,
                                    solver_name, f"{target} (Val)  - {epochs}e")

            # save model
            out_path = (f"./cnn_class/"
                        f"cnn_classifier_{solver_name}_{target}_{epochs}e.h5")
            model.save(out_path)
            print(f"  • saved CNN model → {out_path}")

            #SHAP & importance 
            log_shap_and_importance_classification(
                model=model,
                X_val=X_val_cnn,
                y_val=y_val,
                pred_val=best_pred_val,
                features=features,
                solver_name=solver_name,
                target=target
            )

    
    pd.DataFrame(results).to_csv(
        "./cnn_evaluation_results_classification.csv",
        mode='a', index=False,
        header=not os.path.exists("./cnn_evaluation_results_classification.csv")
    )

In [7]:
def run_all_models(base_folder):
    for root, dirs, files in os.walk(base_folder):
        for folder in dirs:
            folder_path = os.path.join(root, folder)
            csv_files = os.listdir(folder_path)

            train_file = [f for f in csv_files if f.endswith("_train.csv")]
            test_file = [f for f in csv_files if f.endswith("_test.csv")]
            val_file = [f for f in csv_files if f.endswith("_val.csv")]

            if train_file and test_file and val_file:
                train_fp = os.path.join(folder_path, train_file[0])
                test_fp = os.path.join(folder_path, test_file[0])
                val_fp = os.path.join(folder_path, val_file[0])

                solver_name = folder  
                train_cnn_classifier_for_solver(solver_name, train_fp, test_fp, val_fp)

In [ ]:
base_folder = "./trainingData/final_td_min/td_models" #Path to training data
BINS_BASE_DIR = "./trainingData/final_td_min/td_bindata" #Path to bin data
run_all_models(base_folder)


Solver: or_min

=== solution_time → num_classes=2 ===
 train bins: [0 1]
  val bins: [0]
 test bins: [0]

Training CNN for 'solution_time' (50 epochs)...


2025-05-30 00:05:32.172787: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
[TEST]
solution_time (Test) - 50e | Acc: 0.9750, Prec: 0.5000, Rec: 0.4875, F1: 0.4937
[VAL]


solution_time (Val)  - 50e | Acc: 1.0000, Prec: 1.0000, Rec: 1.0000, F1: 1.0000
  • saved CNN model → ./binres_min_kp/cnn_class/cnn_classifier_or_min_solution_time_50e.h5


/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=['Tensor(shape=(560, 24, 1))']
  warnings.warn(msg)
/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=['Tensor(shape=(50, 24, 1))']
  warnings.warn(msg)


...

/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_338
Received: inputs=['Tensor(shape=(560, 24, 1))']
  warnings.warn(msg)
/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_338
Received: inputs=['Tensor(shape=(50, 24, 1))']
  warnings.warn(msg)


Gradient SHAP values and feature importances saved for bb_min-optimality_gap.

Training CNN for 'optimality_gap' (100 epochs)...
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


[TEST]
optimality_gap (Test) - 100e | Acc: 0.8714, Prec: 0.4093, Rec: 0.5931, F1: 0.4721
[VAL]
optimality_gap (Val)  - 100e | Acc: 0.7071, Prec: 0.3220, Rec: 0.4138, F1: 0.3553
  • saved CNN model → ./binres_min_kp/cnn_class/cnn_classifier_bb_min_optimality_gap_100e.h5


/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_351
Received: inputs=['Tensor(shape=(560, 24, 1))']
  warnings.warn(msg)
/home/jovyan/conda-envs/torch-gpu/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_351
Received: inputs=['Tensor(shape=(50, 24, 1))']
  warnings.warn(msg)


Gradient SHAP values and feature importances saved for bb_min-optimality_gap.
  skipping 'peak_memory': only one class in train
